# Lotka-Volterra

![](./lotka_volterra.png)

# Jax Concepts Used

- `jit` -> speed up jax code
- `fori_loop` -> jax native `for-loop` that speeds up computation. Useful when you're `jit`-ing an entire function that has a for-loop. See [exe_03_primitives.ipynb](../../exercises/exe_03_primitives.ipynb) for a further discussion
- `scan` -> like a `fori_loop`, but we can keep and pass along state. 

In [1]:
import numpy as np
import jax.numpy as jnp
import jax
from functools import partial

In [2]:
"""
Notes:
    1) We added in partial function to pass the static_argnames to the jit 
    2) we added in static_argnames to specify that these are constant and should not result in a recompilation
"""
@partial(jax.jit, static_argnames=["alpha", "beta", "gamma", "delta", "dt"])
def _lotka_volterra_step(
    x, y,
    alpha, beta, gamma, delta, dt
):
    
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y
    
    x_new = x + dxdt * dt
    y_new = y + dydt * dt
    
    return x_new, y_new

In [3]:
# Parameters
alpha = 1.1
beta = 0.4
gamma = 0.4
delta = 0.1
dt = 0.1
num_steps = 20


lotka_volterra_step = partial(
    _lotka_volterra_step,
    alpha=alpha, beta=beta, gamma=gamma, delta=delta, dt=dt
)

# Initial populations
x_prev = 10.0
y_prev = 5.0



In [4]:
xs = []
ys = []
for i in range(num_steps):
    x_new, y_new = lotka_volterra_step(
        x_prev, y_prev
    )

    xs.append(x_new)
    ys.append(y_new)

    x_prev = x_new
    y_prev = y_new


In [5]:
for (x, y) in zip(xs, ys):
    print(f"x: {x:.3f}, y: {y:.3f}")


x: 9.100, y: 5.300
x: 8.172, y: 5.570
x: 7.250, y: 5.803
x: 6.365, y: 5.991
x: 5.539, y: 6.133
x: 4.790, y: 6.227
x: 4.124, y: 6.277
x: 3.542, y: 6.284
x: 3.041, y: 6.256
x: 2.615, y: 6.196
x: 2.254, y: 6.110
x: 1.951, y: 6.003
x: 1.698, y: 5.880
x: 1.485, y: 5.745
x: 1.307, y: 5.600
x: 1.158, y: 5.449
x: 1.033, y: 5.295
x: 0.928, y: 5.137
x: 0.839, y: 4.980
x: 0.764, y: 4.822


# fori-loop

We need to redefine our lotka_volterra function to make it compatible with jax. There are two things we can do:

1) get only the last value
2) get the trajectory

## `fori_loop`: Only getting the last value

In [6]:
@partial(jax.jit, static_argnames=["alpha", "beta", "gamma", "delta", "dt"])
def _lotka_volterra_step(
    _, state,
    alpha, beta, gamma, delta, dt
):
    x, y = state 
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y
    
    x_new = x + dxdt * dt
    y_new = y + dydt * dt
    
    return x_new, y_new


lotka_volterra_step = partial(
    _lotka_volterra_step,
    alpha=alpha, beta=beta, gamma=gamma, delta=delta, dt=dt
)

# Initial populations
x_prev = 10.0
y_prev = 5.0

# If we're only concerned with having the last value, we can do the following
x, y = jax.lax.fori_loop(
    lower=0, upper=num_steps,
    body_fun=lotka_volterra_step, init_val=((x_prev, y_prev))
)
print(f"x: {x:.3f}, y: {y:.3f}")


x: 0.764, y: 4.822


## `fori_loop`: Getting the trajectory

We store the trajectories as an array that we pass in. Note that this method is likely **slow** because we are manually indexing in. From a functional programming standpoint, what we're doing is less natural, too, as we can more elegantly express this in a `scan`, that we see later


In [7]:
@partial(jax.jit, static_argnames=["alpha", "beta", "gamma", "delta", "dt"])
def _lotka_volterra_step(
        trajectory_idx, state,
        alpha, beta, gamma, delta, dt
):
    x, y = state[trajectory_idx]
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y

    x_new = x + dxdt * dt
    y_new = y + dydt * dt
    
    state = state.at[trajectory_idx + 1].set([x_new, y_new])
    return state


lotka_volterra_step = partial(
    _lotka_volterra_step,
    alpha=alpha, beta=beta, gamma=gamma, delta=delta, dt=dt
)

# Initial populations
x_prev = 10.0
y_prev = 5.0

trajectory = jnp.zeros((num_steps+1, 2))
trajectory = trajectory.at[0].set([x_prev, y_prev])


# If we're only concerned with having the last value, we can do the following
trajectory = jax.lax.fori_loop(
    lower=0, upper=num_steps,
    body_fun=lotka_volterra_step, init_val=(trajectory)
)

for i in range(len(np.asarray(trajectory))):
    x = trajectory[i, 0]
    y = trajectory[i, 1]
    print(f"x: {x:.3f}, y: {y:.3f}")


x: 10.000, y: 5.000
x: 9.100, y: 5.300
x: 8.172, y: 5.570
x: 7.250, y: 5.803
x: 6.365, y: 5.991
x: 5.539, y: 6.133
x: 4.790, y: 6.227
x: 4.124, y: 6.277
x: 3.542, y: 6.284
x: 3.041, y: 6.256
x: 2.615, y: 6.196
x: 2.254, y: 6.110
x: 1.951, y: 6.003
x: 1.698, y: 5.880
x: 1.485, y: 5.745
x: 1.307, y: 5.600
x: 1.158, y: 5.449
x: 1.033, y: 5.295
x: 0.928, y: 5.137
x: 0.839, y: 4.980
x: 0.764, y: 4.822


# Scan

This is similar to the `fori_loop` where we store the state, except we do not need to use `state = state.at[unused_i+1].set([x_new, y_new])` to store the trajectory. 
    
This is a more "natural" way to pass along the information.

Documentation: [jax.lax.scan](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.scan.html)


In [29]:
@partial(jax.jit, static_argnames=["alpha", "beta", "gamma", "delta", "dt"])
def _lotka_volterra_step(
        state, _,
        alpha, beta, gamma, delta, dt
):
    x, y = state
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y

    x_new = x + dxdt * dt
    y_new = y + dydt * dt

    return (x_new, y_new), (x_new, y_new)


lotka_volterra_step = partial(
    _lotka_volterra_step,
    alpha=alpha, beta=beta, gamma=gamma, delta=delta, dt=dt
)

# Initial populations
x_prev = 10.0
y_prev = 5.0

final_state, trajectory = jax.lax.scan(
    f=lotka_volterra_step,
    init=(x_prev, y_prev),
    xs=None,
    length=num_steps
)



In [30]:
np.asarray(final_state)

array([0.76442355, 4.822236  ], dtype=float32)

In [31]:
for (_x, _y) in np.asarray(trajectory).T:
    print(_x, _y)

9.1 5.3
8.171801 5.5703
7.249923 5.802682
6.3646545 5.991265
5.539473 6.1329374
4.7898855 6.227352
4.1236405 6.276541
3.5419528 6.2843018
3.0412197 6.2555165
2.6147778 6.19554
2.2544048 6.109718
1.9514382 6.003067
1.6975118 5.8800907
1.4849771 5.7447023
1.3070946 5.6002216
1.1580743 5.449413
1.0330294 5.2945447
0.92788583 5.137457
0.83927435 4.9796286
0.76442355 4.822236
